In [22]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [23]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    # Tags are: DET - determiner; NN - noun; V - verb
    # For example, the word "The" is a determiner
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 10

{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [28]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        print('Embedding shape', embeds.shape)
        print('view', embeds.view(len(sentence), 1, -1).shape)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        print('out', lstm_out.shape)
        print('out view', lstm_out.view(len(sentence), -1).shape)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        print(tag_scores)
        return tag_scores

In [ ]:
class LSTMClassifier(nn.Module):
    def __init__(self, input_size=700, hidden_size=128, num_classes=20):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)

    
    def forward(self, x):
        lstm_out, _ = self.lstm(x)         
        print('lstm out', lstm_out.shape)
        print('view', lstm_out.view(140, -1).shape)
        out = self.fc(lstm_out.view(140, -1))             
        return out


In [29]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

Embedding shape torch.Size([5, 6])
view torch.Size([5, 1, 6])
out torch.Size([5, 1, 10])
out view torch.Size([5, 10])
tensor([[-1.0426, -1.2051, -1.0561],
        [-0.9888, -1.2684, -1.0593],
        [-1.0811, -1.0777, -1.1382],
        [-1.0560, -1.1355, -1.1059],
        [-1.0211, -1.1381, -1.1413]])
tensor([[-1.0426, -1.2051, -1.0561],
        [-0.9888, -1.2684, -1.0593],
        [-1.0811, -1.0777, -1.1382],
        [-1.0560, -1.1355, -1.1059],
        [-1.0211, -1.1381, -1.1413]])


In [30]:
for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        print('targets shape', targets)
        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

targets shape tensor([0, 1, 2, 0, 1])
Embedding shape torch.Size([5, 6])
view torch.Size([5, 1, 6])
out torch.Size([5, 1, 10])
out view torch.Size([5, 10])
tensor([[-1.0426, -1.2051, -1.0561],
        [-0.9888, -1.2684, -1.0593],
        [-1.0811, -1.0777, -1.1382],
        [-1.0560, -1.1355, -1.1059],
        [-1.0211, -1.1381, -1.1413]], grad_fn=<LogSoftmaxBackward0>)
targets shape tensor([1, 2, 0, 1])
Embedding shape torch.Size([4, 6])
view torch.Size([4, 1, 6])
out torch.Size([4, 1, 10])
out view torch.Size([4, 10])
tensor([[-0.9929, -1.2028, -1.1112],
        [-0.9203, -1.3132, -1.1008],
        [-0.9738, -1.1937, -1.1418],
        [-1.0672, -1.1250, -1.1045]], grad_fn=<LogSoftmaxBackward0>)
targets shape tensor([0, 1, 2, 0, 1])
Embedding shape torch.Size([5, 6])
view torch.Size([5, 1, 6])
out torch.Size([5, 1, 10])
out view torch.Size([5, 10])
tensor([[-1.0491, -1.1722, -1.0786],
        [-0.9974, -1.2313, -1.0810],
        [-1.0901, -1.0457, -1.1636],
        [-1.0663, -1.0979, 